In [10]:
from collections import defaultdict
import numpy as np

In [12]:
def region_lists(grid: np.array):
    """Lists of region indices from grid"""
    regions = defaultdict(list)
    for (i, j), r in np.ndenumerate(grid):
        regions[r].append((i, j))
    return list(regions.values())

In [13]:
grid = np.array([[0,1,1,1,1],
                [0,1,1,1,2],
                [0,0,0,2,2],
                [3,0,3,2,2],
                [3,3,3,4,4]])

star = np.array([[0,1,0,0,0],
                [0,0,0,0,1],
                [0,0,1,0,0],
                [1,0,0,0,0],
                [0,0,0,1,0]])

In [14]:
region_lists(grid)

[[(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 1)],
 [(0, 1), (0, 2), (0, 3), (0, 4), (1, 1), (1, 2), (1, 3)],
 [(1, 4), (2, 3), (2, 4), (3, 3), (3, 4)],
 [(3, 0), (3, 2), (4, 0), (4, 1), (4, 2)],
 [(4, 3), (4, 4)]]